In [20]:
print("OK")

OK


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone as lp
from langchain.document_loaders import PyMuPDFLoader ,DirectoryLoader
from langchain.document_loaders import PyPDFLoader ,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents



#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks



#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
extracted_data = load_pdf("/Users/badboii/Documents Local/git/GenAI-MedChat/ChatBot/data")

In [12]:
embeddings = download_hugging_face_embeddings()

No sentence-transformers model found with name meta-llama/Llama-2-7b-chat-hf. Creating a new one with mean pooling.


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-67beb4e6-02d7175b3ec695bc27382602;8756961f-c1ed-41fc-9b51-ea6e8ba07312)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
chunks = text_split(extracted_data)

In [ ]:
from pinecone import Pinecone,ServerlessSpec

In [ ]:
pc = Pinecone(api_key="pcsk_2j2M5c_9JsdtV5d6kiUUyc66ZjH634yyzheGp35oirmtWnc9yAyxYJJP4fSFhMz5qefeF")

In [ ]:
index = "medicalchatb"

In [ ]:
chunks[0]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': '/Users/badboii/Documents Local/git/GenAI-MedChat/ChatBot/data/Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')

In [ ]:
import time

index_name = "medicalchatb"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
index

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4


In [ ]:
uuids = [str(uuid4()) for _ in range(len(chunks))]

vector_store.add_documents(documents=chunks, ids=uuids)

In [ ]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import pinecone

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
llm_pipeline = pipeline(
    "text-generation",
    model="sentence-transformers/all-MiniLM-L6-v2",  # Choose any HF model
    torch_dtype="auto",
    device_map="auto",
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [ ]:
qa = RetrievalQA(llm=llm, retriever=retriever)
